In [1]:
from bs4 import BeautifulSoup
import requests
import json


books = []

In [2]:
def get_page(active_page = 1):
    # r = requests.get(f'https://www.dr.com.tr/CokSatanlar/Kitap#/page={active_page}/sort=groups.group.displayorder,asc/categoryid=0/clog=4020/parentId=0/price=-1,-1/hs=false')
    # r = requests.get(f'https://www.kitapyurdu.com/index.php?route=product/best_sellers&list_id=5&limit=100&page={active_page}')

    url = "https://www.dr.com.tr/Catalog/CatalogProducts"
    data = {
        "catalogId": 4020,
        "page": active_page,
        "sortfield": "soldcount",
        "sortorder": "desc",
        "size": 60,
        "categoryid": 0,
        "parentId": 0,
        "mediatypes": "",
        "HideNotForSale": True,
        "minPrice": -1,
        "maxPrice": -1,
        "writer": "",
        "minDiscount": -1,
        "maxdiscount": -1,
        "language": "",
    }

    r = requests.post(url, data)
    soup = BeautifulSoup(r.content,"lxml")
    return soup

In [3]:
def get_detail_page(url):
    r = requests.get(f'https://www.dr.com.tr{url}')
    soup = BeautifulSoup(r.content,"lxml")
    return soup

In [4]:
def get_books(html):
    page_books = html.find_all("div", attrs={"class":"prd-content"})

    for book in page_books:
        product = {}
        product["name"] = book.find("a", attrs={"class":"prd-name"}).text
        product["author"] = book.find("div", attrs={"class":"pName"}).text
        product["price"] = book.find("div", attrs={"class":"prd-price"}).text.replace("\n","").replace("TL","").strip()
        product["publisher"] = book.find("a", attrs={"class":"prd-publisher"}).get("title")
        detail_url = book.find("a", attrs={"class":"prd-name"}).get("href")
        detail_page = get_detail_page(detail_url)
        other_details = get_other_details(detail_page)
        product = product | other_details
        print("Son gelen kitap =>", product)
        books.append(product)

In [5]:
def get_other_details(html):
    details = {}
    rate_area_title = html.find("div", attrs={"class":"prd-rate-title"})
    short_properties = html.find_all("div", attrs={"class":"short-prop"})

    comment_count = rate_area_title.find("span", attrs={"class":"cursor-pointer font-weight-bold fs-2 js-anchor-button"})
    if comment_count is not None:
        details["comment_count"] = comment_count.text.replace("Yorum","").strip()
    else:
        details["comment_count"] = "0"

    star_count = rate_area_title.find("span", attrs={"class":"fs-2 mx-10"}).text
    star_count = star_count.split("-")
    details["star_count"] = star_count[1].replace("Kişi)","").strip()
    details["star_average"] = star_count[0].split("/")[0].replace("(","").strip()

    # details["print_product"] = short_properties[0].text.split(":")[1].strip()
    # details["page_numbers"] = short_properties[1].text.split(":")[1].strip()

    return details


In [6]:
books = []
get_books(get_page(1))
print(books[0])
print(books[59])

Son gelen kitap => {'name': 'Esra Ezmeci Seti 5 Kitap Takım - Defter Hediyeli', 'author': 'Esra Ezmeci', 'price': '139,20', 'publisher': 'Destek Yayınları', 'comment_count': '6', 'star_count': '72', 'star_average': '3,0'}
Son gelen kitap => {'name': 'İz Bıraktığın Kadar Varsın', 'author': 'Esra Ezmeci', 'price': '32,40', 'publisher': 'Destek Yayınları', 'comment_count': '6', 'star_count': '582', 'star_average': '3,0'}
Son gelen kitap => {'name': 'Yaşamak Sakinlik İster', 'author': 'Özgür Bacaksız', 'price': '22,00', 'publisher': 'Destek Yayınları', 'comment_count': '12', 'star_count': '56', 'star_average': '3,5'}
Son gelen kitap => {'name': 'Sen Yola Çık Yol Sana Görünür', 'author': 'Hakan Mengüç', 'price': '19,20', 'publisher': 'Destek Yayınları', 'comment_count': '7', 'star_count': '35', 'star_average': '3,5'}
Son gelen kitap => {'name': 'Duygusal Zeka', 'author': 'Bircan Yıldırım', 'price': '28,80', 'publisher': 'Destek Yayınları', 'comment_count': '63', 'star_count': '296', 'star_a

In [7]:
books = []
print("büyük işlem")
for i in range(10):
    print(i)
    get_books(get_page(i))

print("Books length:", len(books));
print("büyük işlem bitti!")

Büyük operasyon başlıyor!!!
0
Son gelen kitap => {'name': 'Esra Ezmeci Seti 5 Kitap Takım - Defter Hediyeli', 'author': 'Esra Ezmeci', 'price': '139,20', 'publisher': 'Destek Yayınları', 'comment_count': '6', 'star_count': '72', 'star_average': '3,0'}
Son gelen kitap => {'name': 'İz Bıraktığın Kadar Varsın', 'author': 'Esra Ezmeci', 'price': '32,40', 'publisher': 'Destek Yayınları', 'comment_count': '6', 'star_count': '582', 'star_average': '3,0'}
Son gelen kitap => {'name': 'Yaşamak Sakinlik İster', 'author': 'Özgür Bacaksız', 'price': '22,00', 'publisher': 'Destek Yayınları', 'comment_count': '12', 'star_count': '56', 'star_average': '3,5'}
Son gelen kitap => {'name': 'Sen Yola Çık Yol Sana Görünür', 'author': 'Hakan Mengüç', 'price': '19,20', 'publisher': 'Destek Yayınları', 'comment_count': '7', 'star_count': '35', 'star_average': '3,5'}
Son gelen kitap => {'name': 'Duygusal Zeka', 'author': 'Bircan Yıldırım', 'price': '28,80', 'publisher': 'Destek Yayınları', 'comment_count': '63'

In [8]:
books_json = json.dumps(books, indent = 2, ensure_ascii = False)
print(books_json)

[
  {
    "name": "Esra Ezmeci Seti 5 Kitap Takım - Defter Hediyeli",
    "author": "Esra Ezmeci",
    "price": "139,20",
    "publisher": "Destek Yayınları",
    "comment_count": "6",
    "star_count": "72",
    "star_average": "3,0"
  },
  {
    "name": "İz Bıraktığın Kadar Varsın",
    "author": "Esra Ezmeci",
    "price": "32,40",
    "publisher": "Destek Yayınları",
    "comment_count": "6",
    "star_count": "582",
    "star_average": "3,0"
  },
  {
    "name": "Yaşamak Sakinlik İster",
    "author": "Özgür Bacaksız",
    "price": "22,00",
    "publisher": "Destek Yayınları",
    "comment_count": "12",
    "star_count": "56",
    "star_average": "3,5"
  },
  {
    "name": "Sen Yola Çık Yol Sana Görünür",
    "author": "Hakan Mengüç",
    "price": "19,20",
    "publisher": "Destek Yayınları",
    "comment_count": "7",
    "star_count": "35",
    "star_average": "3,5"
  },
  {
    "name": "Duygusal Zeka",
    "author": "Bircan Yıldırım",
    "price": "28,80",
    "publisher": "Dest